In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms, datasets, models
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc

# DataLoader for loading and augmenting images
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import datasets
# Early Stopping implementation
import time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
# Defining the trransfromation steps data aug ,resizing, normalization
train_transforms = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet mean/std
])
test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Loading the data from the train and test directories
train_dataset = datasets.ImageFolder(root='/kaggle/input/dataset/data/train',transform=train_transforms)
print(f"Total training images: {len(train_dataset)}")
val_dataset = datasets.ImageFolder(root='/kaggle/input/dataset/data/val',transform=train_transforms)
print(f"Total val images: {len(val_dataset)}")
test_dataset = datasets.ImageFolder(root='/kaggle/input/dataset/data/test', transform=test_transforms)
print(f"Total test images: {len(test_dataset)}")

Total training images: 4192
Total val images: 1040
Total test images: 624


In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32,shuffle=False)
print(f"Train_loader:{len(train_loader)}, val_loader:{len(val_loader)}, test_loader:{len(test_loader)}")

Train_loader:131, val_loader:33, test_loader:20


In [6]:
print(train_loader.dataset.class_to_idx)

{'normal': 0, 'opacity': 1}


In [ ]:

listx = []
for batch in train_loader:
    images, labels = batch  # 'labels' is a batch of labels, not a single label
    for label in labels:  # Iterate over individual labels in the batch
        if label.item() == 0:  # Convert tensor to scalar value using .item()
            listx.append("Pneumonia")
        else:
            listx.append("Normal")

# Ensure that listx is properly structured and passed as input
if len(listx) > 0:
    sns.countplot(x=listx)  # Specify x to avoid errors
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.title("Class Distribution")
    plt.show()
    plt.savefig('training_validation_loss_accuracy.png')
else:
    print("List of labels is empty.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()

        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # Batch Normalization
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)  # Batch Normalization
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Second convolutional block
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  # Batch Normalization
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)  # Batch Normalization
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.dropout = nn.Dropout(0.5)  # Dropout layer for regularization

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 32 * 32, 512)  # Adjust according to the input size
        self.fc2 = nn.Linear(512, 2)  # Binary classification (2 outputs)

    def forward(self, x):
        # First convolutional block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)

        # Second convolutional block
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)

        # Flatten the feature map
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(x)  # Apply dropout

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Logits for binary classification

        return x

# Step 2: Define your model (assuming 'model' is your defined CNN)
model = PneumoniaCNN() 
model = model.to(device)  # Move the model to GPU

# Initialize the model, loss function, and optimizer
#model = PneumoniaCNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# Define your model, criterion, and optimizer
model = PneumoniaCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Mymodel.parameters(), lr=0.001)

# Initialize the StepLR scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Adjust step_size and gamma as needed

In [ ]:

# Assuming model, train_loader, val_loader, criterion, and optimizer are already defined
def evaluate_train(train_loader,val_loader,model,num_epochs):
    # Define the number of epochs
    #num_epochs = 10

    correct=0
    total =0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    # Start training

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        for images, labels in train_loader:  # Iterate over the training data
            images, labels = images.to(device), labels.to(device)
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)  # Get predictions from the model

            # Calculate the loss
            loss = criterion(outputs, labels)  # Compare predictions to true labels

            # Backward pass
            loss.backward()  # Compute gradients

            # Update the weights
            optimizer.step()  # Adjust the model parameters

            # Accumulate the loss for logging
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate average loss for the epoch
        avg_loss = running_loss / len(train_loader)

        train_accuracy = 100 * correct / total
        train_losses.append(avg_loss)
        train_accuracies.append(train_accuracy)
        print(f"Epoch [{epoch + 1}/{num_epochs}], \n Train_Loss: {avg_loss:.4f}, Train_accuracy:{train_accuracy} ")

        # (Optional) Validate the model
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient computation
            val_loss = 0.0
            correct = 0
            total = 0
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                val_outputs = model(val_images)
                val_loss += criterion(val_outputs, val_labels).item()
                _, predicted = torch.max(val_outputs.data, 1)  # Get predicted classes
                total += val_labels.size(0)  # Total number of samples
                correct += (predicted == val_labels).sum().item()  # Count correct predictions


            avg_val_loss = val_loss / len(val_loader)
            val_accuracy = 100 * correct / total
            val_losses.append(avg_val_loss)
            val_accuracies.append(val_accuracy)
            print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
    return train_losses,train_accuracies,val_losses,val_accuracies

In [ ]:
def plot_train_val(train_losses,train_accuracies,val_losses,val_accuracies):
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss over eposchs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()

    plt.savefig('training_validation_loss_accuracy.png')  # Save as a PNG file
    plt.show()

    plt.close()

In [ ]:
def confusion_matrix_plot(test_loader,model):
# Assuming you have your model evaluation loop to get predictions
    predicted = []
    all_labels = []

    # Model evaluation
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for images, labels in test_loader:  # Replace with your data loader
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predicted.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, predicted)
    tn, fp, fn, tp = cm.ravel()  # Extract values


    # Print TP, FP, TN, FN
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"True Negatives: {tn}")
    print(f"False Negatives: {fn}")

    # Normalize the confusion matrix to get percentages
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])

    # Add TP and FN text to the heatmap
    plt.text(0.5, 0.3, f'True Negative', ha='center', va='center', color='black', fontsize=12)
    plt.text(0.5, 1.3, f'False Negative', ha='center', va='center', color='black', fontsize=12)
    plt.text(1.5, 0.3, f'False Positive', ha='center', va='center', color='black', fontsize=12)
    plt.text(1.5, 1.3, f'True Positive', ha='center', va='center', color='white', fontsize=12)

    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix with TP, FN, FP, TN')
    plt.savefig('confusion_matrix_with_metrics.png')  # Save confusion matrix as an image
    plt.show()

In [ ]:
def plot_roc_auc(test_loader,model):
    # Collect predictions and true labels
    all_labels = []
    all_probs = []

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        for test_images, test_labels in test_loader:
            test_images, test_labels = test_images.to(device), test_labels.to(device)
            test_outputs = model(test_images)

            # Get probabilities for the positive class (assuming class 1 is pneumonia)
            test_probs = torch.softmax(test_outputs, dim=1)[:, 1].cpu().numpy()

            all_labels.extend(test_labels.cpu().numpy())  # Collect labels
            all_probs.extend(test_probs)  # Collect probabilities

    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Calculate ROC AUC
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig("roc_curve_train_test.png")  # Save the ROC curve as a PNG file
    plt.show()  # Display the plot
    print(f"AUC:{roc_auc * 100:.2f}%")

In [ ]:
# Overfitting to a small subset
small_dataset_size = 10  # Choose a small number of samples for overfitting
small_dataset = Subset(train_dataset, list(range(small_dataset_size)))  # Select first N samples

# Create data loader for small dataset
small_train_loader = DataLoader(small_dataset, batch_size=2, shuffle=True)
evaluate_train(small_train_loader,val_loader,Mymodel,num_epochs=5)

In [ ]:
evaluate_train(train_loader,val_loader,Mymodel,num_epochs=5)

In [ ]:
confusion_matrix_plot(test_loader)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
# Load pre-trained ResNet-50 model
resnet = models.resnet50(pretrained=True)
num_classes = 2  # Assuming "normal" and "opacity"
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(device)
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)  # Adjust the learning rate as needed
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # Adjust step_size and gamma as needed

# Set the model to evaluation mode (important for inference)
resnet.eval()
train_losses,train_accuracies,val_losses,val_accuracies=evaluate_train(train_loader,val_loader,model=resnet,num_epochs=10)

In [ ]:
plot_train_val(train_losses,train_accuracies,val_losses,val_accuracies)

In [ ]:
# Assuming 'model' is your trained model

# Save the model in HDF5 format
Mymodel.save('pneumonia_model.h5')
print("Model saved to pneumonia_model.h5")

In [ ]:
# Save the entire model
torch.save(model, 'pneumonia_cnn_model.pth')
print("Model saved as pneumonia_cnn_model.pth")